In [1]:
import geopandas as gpd
import fiona

## gpkg files

Geopackage files are sqlite databases with layers of geometry information. The gpkg file for Namibia here has been downloaded from [https://gadm.org/download_country.html](https://gadm.org/download_country.html)

In [2]:
gpkg_filename = '../src/deepmap/data/gadm41_NAM.gpkg'

We can use `geopandas` to read a gpkg as a DataFrame.

In [3]:
df = gpd.read_file(gpkg_filename)
df.head()  # A single row since the gpkg is only for a specific country (NAM)

,GID_0,COUNTRY,geometry
0,NAM,Namibia,"MULTIPOLYGON (((16.28153 -28.50514, 16.28181 -..."


We'll use the more convenient `fiona` package to deal with the geometries included in this `.gpkg` file.

A `.gpkg` file has layers. In this case, each layer represents polygons at different administrative/geographical scales (something like country->state->county).

In [4]:
layer_names = fiona.listlayers(gpkg_filename)
print(layer_names)

['ADM_ADM_0', 'ADM_ADM_1', 'ADM_ADM_2']


In [5]:
first_layer_name = layer_names[0]

# A gpkg has layers
with fiona.open(gpkg_filename, layer=first_layer_name) as layer:
    
    i = 0
    
    # A layer has features
    for feature in layer:
        # A feature has geometry (and other properties)
        geometry = feature['geometry']
        
        # A geometry has a type and coordinates
        if geometry['type'] == 'MultiPolygon':
            coordinates = geometry['coordinates']
            
            # coordinates is a list of 1 (not sure why 1 extra level of abstraction here)
            polygons = coordinates[0]
            
            # For each polygon
            for polygon in polygons:
                
                # For each vertex in the polygon
                for coordinate in polygon:
                    
                    # The vertex is a 2-tuple of coordinates
                    assert type(coordinate) == tuple
                    assert len(coordinate) == 2